# Treinamento do Modelo Two-Tower (Retrieval)
Neste notebook, definimos e treinamos nossa rede neural de arquitetura **Two-Tower**.
Esta arquitetura é padrão da indústria para a etapa de *Retrieval* (Geração de Candidatos), pois permite indexar os vetores de itens e realizar buscas ultra-rápidas (ANN) em tempo real.

## 1. Definição do Dataset e Modelo

### Dataset (`RecSysDataset`)
Carrega os dados de treino e prepara os tensores para o PyTorch.

### Arquitetura Two-Tower (`TwoTowerModel`)
O modelo consiste em duas redes neurais separadas (torres):
1.  **User Tower**: Recebe ID do usuário + Features de contexto -> Gera vetor de usuário (Query).
2.  **Item Tower**: Recebe ID do item + Features do item -> Gera vetor de item (Candidate).

**Loss Function**: Utilizamos **In-Batch Negatives**. Para cada par positivo (usuário, item) no batch, consideramos todos os outros itens do mesmo batch como exemplos negativos. Isso é eficiente e evita a necessidade de amostragem negativa manual.

In [ ]:
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import json
import sys

sys.path.append('./src')

from dataset import RecSysDataset
from model import TwoTowerModel

## 2. Configuração do Treinamento
Carregamos os metadados (número total de usuários e itens) para inicializar as camadas de Embedding com o tamanho correto.
Preparamos o `DataLoader` para fornecer batches de dados para a GPU.

In [2]:
with open("./data/model_metadata.json", "r") as f:
    meta = json.load(f)
    
print(f"🚀 Iniciando Treino Two-Tower. Users: {meta['num_users']}, Items: {meta['num_items']}")

dataset = RecSysDataset("./data/training_dataset.parquet")
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True, num_workers=4)

model = TwoTowerModel(num_users=meta['num_users'], num_items=meta['num_items'])

print(model)

🚀 Iniciando Treino Two-Tower. Users: 14761, Items: 8451
TwoTowerModel(
  (user_embedding): Embedding(14761, 32)
  (user_mlp): Sequential(
    (0): Linear(in_features=34, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
  )
  (item_embedding): Embedding(8451, 32)
  (item_mlp): Sequential(
    (0): Linear(in_features=34, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
  )
)


## 3. Execução do Treinamento
Utilizamos o **PyTorch Lightning** para gerenciar o loop de treinamento.
Treinamos por 5 épocas e salvamos o checkpoint do modelo treinado.

In [3]:
trainer = pl.Trainer(max_epochs=5, accelerator="gpu", devices=1)
trainer.fit(model, dataloader)

print("✅ Modelo Treinado! Salvando artefatos...")
trainer.save_checkpoint("./data/two_tower_model.ckpt")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type       | Params | Mode  | FLOPs
--------------------------------------------------------------
0 | user_embedding | Embedding  | 472 K  | train | 0    
1 | user_mlp       | Sequential | 4.3 K  | train | 0    
2 | item_embedding | Embedding  | 270 K  | train | 0    
3 | item_mlp       | Sequential | 4.3 K  | train | 0    
--------------------------------------------------------------
751 K     Trainable params
0         Non-trainable params
751 K     Total params
3.006     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode
0         Total Flops
/usr/local/lib/python3.10/dist-packages/pytorc

Epoch 4: 100%|██████████| 15/15 [00:01<00:00, 13.21it/s, v_num=0, train_loss=6.050]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 15/15 [00:01<00:00,  9.12it/s, v_num=0, train_loss=6.050]

`weights_only` was not set, defaulting to `False`.



✅ Modelo Treinado! Salvando artefatos...
